# Lab 6.3: Spark Join Demo

## Tổng quan bài tập
**Đề bài**: Hãy hoàn thiện các phần `[...]` để hoàn thiện đoạn code và giải quyết bài toán theo yêu cầu. Ở bài Lab này, bạn sẽ được thực hành thao tác Join hai Dataframe để trích xuất dữ liệu sử dụng Spark.




# Cài đặt Spark trên Google Colab

Để có thể sử dụng Spark trên môi trường Google Colab thì bạn sẽ cần cài đặt một số thành phần sau:
- Java 8
- Spark Binary
- findspark

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,333 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main Sources [2,268 kB]
Get:13 http://archive.ubuntu.com/ubuntu 

Sau đó, bạn sẽ cần khai báo cho hệ thống các đường dẫn cho các thành phần vừa cài.

In [2]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

findspark.init()

# Spark Join Demo

Bạn sẽ cần khởi tạo 1 SparkSesson để có thể bắt đầu Spark.

In [5]:
from pyspark import SparkConf
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as f

conf = SparkConf() \
    .setMaster('local') \
    .setAppName('Lab6.3')

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext

Tạo Dataframe `order_df` chứa thông tin về các đơn đặt hàng.

In [7]:
orders_list = [("01", "02", 350, 1),
                ("01", "04", 580, 1),
                ("01", "07", 320, 2),
                ("02", "03", 450, 1),
                ("02", "06", 220, 1),
                ("03", "01", 195, 1),
                ("04", "09", 270, 3),
                ("04", "08", 410, 2),
                ("05", "02", 350, 1)]

# Tạo Dataframe
order_df = spark.createDataFrame(orders_list, "order_id: string, prod_id: string, unit_price: int, qty: int" )
order_df.show()

+--------+-------+----------+---+
|order_id|prod_id|unit_price|qty|
+--------+-------+----------+---+
|      01|     02|       350|  1|
|      01|     04|       580|  1|
|      01|     07|       320|  2|
|      02|     03|       450|  1|
|      02|     06|       220|  1|
|      03|     01|       195|  1|
|      04|     09|       270|  3|
|      04|     08|       410|  2|
|      05|     02|       350|  1|
+--------+-------+----------+---+



Tạo Dataframe `product_list` chứa thông tin về các sản phẩm.

In [8]:
product_list = [("01", "Scroll Mouse", 250, 20),
                ("02", "Optical Mouse", 350, 20),
                ("03", "Wireless Mouse", 450, 50),
                ("04", "Wireless Keyboard", 580, 50),
                ("05", "Standard Keyboard", 360, 10),
                ("06", "16 GB Flash Storage", 240, 100),
                ("07", "32 GB Flash Storage", 320, 50),
                ("08", "64 GB Flash Storage", 430, 25)]

# Tạo Dataframe
product_df = spark.createDataFrame(product_list, "prod_id: string, prod_name: string, list_price: int, qty: int")
product_df.show()

+-------+-------------------+----------+---+
|prod_id|          prod_name|list_price|qty|
+-------+-------------------+----------+---+
|     01|       Scroll Mouse|       250| 20|
|     02|      Optical Mouse|       350| 20|
|     03|     Wireless Mouse|       450| 50|
|     04|  Wireless Keyboard|       580| 50|
|     05|  Standard Keyboard|       360| 10|
|     06|16 GB Flash Storage|       240|100|
|     07|32 GB Flash Storage|       320| 50|
|     08|64 GB Flash Storage|       430| 25|
+-------+-------------------+----------+---+



Thực hiện thao tác Join trên hai Dataframe

In [10]:
# Xác định điều kiện Join
join_expr = order_df.prod_id == product_df.prod_id

product_renamed_df = product_df.withColumnRenamed("qty", "reorder_qty")

# Join hai Dataframe
order_df.join(product_renamed_df, join_expr, "inner") \
    .drop(product_renamed_df.prod_id) \
    .select("order_id", "prod_id", "prod_name", "unit_price", "list_price", "qty") \
    .show()

+--------+-------+-------------------+----------+----------+---+
|order_id|prod_id|          prod_name|unit_price|list_price|qty|
+--------+-------+-------------------+----------+----------+---+
|      03|     01|       Scroll Mouse|       195|       250|  1|
|      01|     02|      Optical Mouse|       350|       350|  1|
|      05|     02|      Optical Mouse|       350|       350|  1|
|      02|     03|     Wireless Mouse|       450|       450|  1|
|      01|     04|  Wireless Keyboard|       580|       580|  1|
|      02|     06|16 GB Flash Storage|       220|       240|  1|
|      01|     07|32 GB Flash Storage|       320|       320|  2|
|      04|     08|64 GB Flash Storage|       410|       430|  2|
+--------+-------+-------------------+----------+----------+---+

